# Generating Text from Nietzche's Writings

In [1]:
'''
#Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


Using TensorFlow backend.


## Get dataset

In [2]:

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


corpus length: 600893
total chars: 57


## Truncate data

In [3]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


## Vectorize data

In [4]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


## Build model

In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

W0816 07:12:13.575131 12040 deprecation_wrapper.py:119] From C:\Anaconda3\envs\sofenn\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 07:12:13.611110 12040 deprecation_wrapper.py:119] From C:\Anaconda3\envs\sofenn\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 07:12:13.624103 12040 deprecation_wrapper.py:119] From C:\Anaconda3\envs\sofenn\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Build model...


W0816 07:12:14.067849 12040 deprecation_wrapper.py:119] From C:\Anaconda3\envs\sofenn\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0816 07:12:14.083839 12040 deprecation_wrapper.py:119] From C:\Anaconda3\envs\sofenn\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


## Generate texts using callback functions

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
200285/200285 [==============================] - 117s 583us/step - loss: 1.6003s - los

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "er
world beyond the surface, an abyss be"
er
world beyond the surface, an abyss be manifolst to the sense of the self-delight to the present of the painter the present of the self-to the self-decession of the sense of the self-derstand in the self-respect of the strength of the same to the sufferent of the spirit of the happiness of the present to the regard to the free the present the more the portical the manifestions to the dections to the sense of the present of the self-de
----- diversity: 0.5
----- Generating with seed: "er
world beyond the surface, an abyss be"
er
world beyond the surface, an abyss betrain and may be made to the last the self-readical mannst every as a stands of the fact the cause of the prividitions the presses it would is the suffered the more impulses the present of the intert

for, less that supertare and the blear to refiled. but other greaterful a
turment i each just as us as negent that wong: the first" nalitime lass, conditions: once is did
we there a dicclount ourselves--the problem of ca? this justide, and trifity and ethics. they shall
through upon say, almost themselves areing power hearr.
rystesp me fundance, and ge(ther this not aga
----- diversity: 1.2
----- Generating with seed: "to small states, and above
all the intro"
to small states, and above
all the introcease and liquc of that bedominess. they thinking!red than there is "misuth, in such himhicness"" ectived,
our lemence" for framel, in tominal to glowth,
.


we tuct to
law, will
after such ad
iderfors, to avents,--thike comprehens life
espitable bid struigfrous, bedocks." like
the
unvigorabully,
mirrou
g degenialily illker, to
? she worls in.

for ounder there.ares to their elogs that do will to 
Epoch 5/20
200285/200285 [==============================] - 111s 556us/step - loss: 1.4355

--

feel the party and and and as the concerned the man as the present of the moral conscience of the subject of the subject, and as the power the compulsous and and as the moral and a present of the moral and the man is as the more among the sense--and and as the subject, and as the present the subject of the other in the probably and the sense, the person is the course of the concerning and as the compul
----- diversity: 0.5
----- Generating with seed: "e a strong imagination in order to
feel "
e a strong imagination in order to
feel the deceived at an action that the own will to the relation that the capacis to be about the more peruages and and the artifility of the consequently the among the world of such i in the same as passion is the concepted the invirture leight at the reason in spirit of the originated and the experiences of the means and the fingle of its preferted taken which is a specised and concemance of man of t
----- diversity: 1.0
----- Generating with seed: "e a strong 

sufface; which is a lies in the is the most scall, buinden esidedne t
----- diversity: 1.2
----- Generating with seed: " mastery. but she does not want truth--w"
 mastery. but she does not want truth--whowe happensness" those woman asting, a quality taste the
"apleadet,
stanged in which is bur; disgustss"
of unorures. mentangor of work the cost ocer refined. pralous in all religion--mistophic "orignihim! has the bijogn unglooms,
other
no new spras phaseity
whethertoverial heart
der; we isteruply sisposs,
furtherwfarotaces, excasion ascenised the moralism: is allege, owibjuging repeative kind of 
Epoch 12/20
200285/200285 [==============================] - 114s 569us/step - loss: 1.3659

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "est counter-arguments. occasionally,
the"
est counter-arguments. occasionally,
the sensual the conceptions and and and and such and and presential of the conscience of the strength of the such a subjection of an acti